<a href="https://colab.research.google.com/github/TuncerCemUgurluer/EEG_deception_detection/blob/code/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import numpy as np
import shutil

In [ ]:
 eeg_channels = ["EEG.AF3", "EEG.F7", "EEG.F3", "EEG.FC5", "EEG.T7", "EEG.P7", "EEG.O1", "EEG.O2", "EEG.P8", "EEG.T8", "EEG.FC6", "EEG.F4", "EEG.F8", "EEG.AF4"]


In [ ]:
def augment_frequency_domain(signal, perturbation_factor=0.05):
    # Transform the signal to the frequency domain
    freq_domain = np.fft.fft(signal)

    # Get magnitude and phase
    magnitude = np.abs(freq_domain)
    phase = np.angle(freq_domain)

    # Introduce perturbations to magnitude
    magnitude_perturbed = magnitude + perturbation_factor * np.random.normal(0, 1, magnitude.shape)

    # Reconstruct frequency domain signal with perturbed magnitude
    freq_domain_perturbed = magnitude_perturbed * np.exp(1j * phase)

    # Transform back to time domain
    augmented_signal = np.real(np.fft.ifft(freq_domain_perturbed))

    return augmented_signal

In [ ]:
def augment_jittering(signal, noise_factor=0.01):
    """
    Add Gaussian noise to the signal.
    """
    noise = noise_factor * np.random.normal(size=signal.shape)
    return signal + noise

In [ ]:
def augment_and_copy_eeg_file(filepath, category, save_dir):
    """
    Apply both frequency-domain and jittering augmentations to an EEG file,
    save the augmented versions in the specified category folder,
    and copy the original file to the save directory.
    """
    # Load only the necessary columns
    data = pd.read_csv(filepath, usecols=eeg_channels)

    # Determine the save subdirectory based on category
    save_subdir = os.path.join(save_dir, category)
    if not os.path.exists(save_subdir):
        os.makedirs(save_subdir)

    # Frequency-domain augmentation
    data_frequency = data.copy()
    for channel in eeg_channels:
        data_frequency[channel] = augment_frequency_domain(data[channel].values)
    base_filename = os.path.basename(filepath)
    augmented_filename_frequency = base_filename.split(".")[0] + "_frequency_aug.csv"
    augmented_filepath_frequency = os.path.join(save_subdir, augmented_filename_frequency)
    data_frequency.to_csv(augmented_filepath_frequency, index=False)

    # Jittering augmentation
    data_jittering = data.copy()
    for channel in eeg_channels:
        data_jittering[channel] = augment_jittering(data[channel].values)
    augmented_filename_jittering = base_filename.split(".")[0] + "_jittering_aug.csv"
    augmented_filepath_jittering = os.path.join(save_subdir, augmented_filename_jittering)
    data_jittering.to_csv(augmented_filepath_jittering, index=False)

    # Copy the original file to the save subdirectory
    shutil.copy(filepath, os.path.join(save_subdir, base_filename))

In [ ]:
lie_dir = "/content/drive/MyDrive/EEG/Serkan_new/Serkan_new_Lie"
truth_dir = "/content/drive/MyDrive/EEG/Serkan_new/Serkan_new_True"
save_dir = "/content/drive/MyDrive/EEG/augmented_data"

In [ ]:
for filepath in os.listdir(lie_dir):
    augment_and_copy_eeg_file(os.path.join(lie_dir, filepath), "lie", save_dir)

In [ ]:
for filepath in os.listdir(truth_dir):
    augment_and_copy_eeg_file(os.path.join(truth_dir, filepath),"truth", save_dir)